In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

In [3]:
class Equipamento:
    def __init__(self, 
                 tipo:str,
                 marca:str,
                 modelo:str,
                 local:str, #Local de instalação
                 tensao:int =None,
                 dc:bool = True,#Tipo de alimentação Corrente Continua ou Alternada, por padrão Continua DC 
                 instalador=None,
                 data_compra=None,
                 temp_garantia:int=3, #Tempo de garantia em meses
                 data_instalacao=None,):
        
        self._tipo = tipo
        self._marca = marca
        self._modelo = modelo
        self._local = local
        self._tensao = tensao
        self._dc = dc
        self._temp_garantia = temp_garantia

        if instalador:
            self._instalador = instalador
        else:
            self._instalador = None

        if data_instalacao:
            self._data_instalacao = datetime.strptime(data_instalacao, "%d-%m-%Y")
        else:
            self._data_instalacao = None

        if data_compra:
            self._data_compra = datetime.strptime(data_compra, "%d-%m-%Y")
        else:
            self._data_compra = None
        
        self._data_ultimo_defeito = None
        self._ultimo_defeito = None
        self._historico_manutencao = pd.DataFrame(columns=["Data", "Tec","Defeito"])

    #Getters e Setters
    def getTipoEquipamento(self):
        return self._tipo
    
    def getMarca(self):
        return self._marca
    
    def getModelo(self):
        return self._modelo
    
    def getLocal(self):
        return self._local
    def _setLocal(self,local):
        self._local = local

    def getTensao(self):
        return self._tensao
    def _setTensao(self,tensao):
        self._tensao = tensao

    def isDC(self):
        return self._dc
    
    def getTempGarantia(self):
        return self._temp_garantia
    
    def getInstalador(self):
        return self._instalador
    
    def _setInstalador(self,instalador):
        self._instalador = instalador
    
    def getCompra(self):
        if self._data_compra: 
            return self._data_compra
        else:
            return f"Não foi definida a data de compra do(a) {self.getTipoEquipamento()}"
    
    def getInstalacao(self):
        if self._data_instalacao:
            return self._data_instalacao
        else:
            return f"Não foi definida a data de instalação do(a) {self.getTipoEquipamento()}"
        
    def getDataUltimaManutencao(self):
        if self._data_ultima_manutencao:
            return self._data_ultima_manutencao
        else:
            return "não houve manutenções ainda"
        
    def getTecUltimaManutencao(self):
        if self._tec_ultima_manutencao:
            return self._tec_ultima_manutencao
        else:
            return "não houve manutenções ainda"
    
    def _setUltimoDefeito(self,data,desc_defeito):
        self._ultimo_defeito = desc_defeito
        self._data_ultima_medicao = data
    
    def IsForaDaGarantia(self):
        if self._data_compra:
            if self._data_compra + relativedelta(months=self.getTempGarantia()) < datetime.now():
                return True
            else:
                return False
        else:
            if self.getTempGarantia():
                return f"Não foi definida a data de compra do(a) {self.getTipoEquipamento()}"
            else:
                return "Não foi informado o tempo de garantia"
    
    def reinstalacao(self,instalador,local,data,tensao=None):
        self._data_instalacao = datetime.strptime(data, "%d-%m-%Y")
        self._setInstalador(instalador)
        self._setLocal(local)
        if tensao:
            self._setTensao(tensao)

    def manutencao(self,tec,data,defeito):
        self._data_ultima_manutencao = datetime.strptime(data, "%d-%m-%Y")
        self._tec_ultima_manutencao = tec
        if defeito:
            self._setUltimoDefeito(self.getDataUltimaManutencao(),defeito)
        nova_medicao = pd.DataFrame({'Data':[self.getDataUltimaManutencao()], 'Tec':[self.getTecUltimaManutencao()], "Defeito":[defeito]})
        self._historico_manutencao = pd.concat([self._historico_manutencao, nova_medicao], ignore_index=True)

    

In [4]:
class Bateria(Equipamento):
    """
    Classe Bateria

    Recebe: Tudo que um equipamento gênerico tem

    Recebe: 
    Tensão de ciclo (tensão que a bateria flutua em funcionamento)

    Tensão de carregamento (tensão que deve ser usada para carregamento da bateria)

    Histórico de manutenção inclui a tensão medida da bateria mesmo que não esteja com defeito
            
    
    """
    def __init__(self, 
                 marca,
                 modelo,
                 local, #Local de instalação
                 medicao_inicial=12.8, 
                 tensao_ciclo=(13.5,13.8), 
                 tensao_carregamento=(14.3,14.8),
                 tensao =None,
                 dc = True,#Tipo de alimentação Corrente Continua ou Alternada, por padrão Continua DC 
                 instalador=None,
                 data_compra=None,
                 temp_garantia=3, #Tempo de garantia em meses
                 data_instalacao=None,
                 ):
        super().__init__(tipo="Bateria",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=None,
                         dc=True,
                         instalador=instalador,
                         data_compra=data_compra,
                         temp_garantia=temp_garantia,
                         data_instalacao=data_instalacao,
                         )
        
        self._tensao_ciclo = tensao_ciclo
        self._tensao_carregamento = tensao_carregamento
        self._tensao_ultima_manutencao = medicao_inicial
        self._data_ultima_manutencao = self._data_instalacao
        if self._data_ultima_manutencao:
            self._prox_medicao = self._data_ultima_manutencao + relativedelta(months=3)
        else:
            self._prox_medicao = None
        self._tensao_baixa = False
        self._historico_manutencao = pd.DataFrame(columns=["Data","Tec","Defeito","Tensao"])

    def __str__(self):
        output=f"""Marca: {self.getMarca()}
Data de Compra: {self.getCompra()}
Data Instalação: {self.getInstalacao()}
Última Medição: {self.getDataUltimaManutencao()}
Tensão de Ciclo: {self.getTensaoCiclo()}
Tensão de Carregamento: {self.getTensaoCarregamento()}"""
        return output

    # Getters and Setters
    def getTensaoCiclo(self):
        """Retorna tensão de ciclo da bateria"""
        return self._tensao_ciclo

    def getTensaoCarregamento(self):
        """Retorna tensão de carregamento da bateria
        """
        return self._tensao_carregamento

    def getTensao(self):
        """Retorna última tensão medida na bateria"""
        return self._tensao_ultima_manutencao

    def IsLowVoltage(self):
        """
        Retorna se a tensão esta baixa
        """
        return self._tensao_baixa

    def getProxManutencao(self):
        """
        Retorna a data da próxima manutenção
        por padrão 3 meses
        """
        if self._prox_manutencao:
            return self._prox_manutencao
        else:
            return "Não há data da última medição"

    def _setTensao(self, tensao):
        if tensao < self.getTensaoCiclo()[0]:
            self._tensao_baixa = True
        self._tensao_ultima_medicao = tensao

    def _proximaManutencao(self): # Adiciona 3 meses à data da última medicão
        self._prox_manutencao = self._data_ultima_manutencao + relativedelta(months=3)
        return self._prox_manutencao

    def manutencao(self, tec, data:str, tensao:float, defeito:str):
        """
        Registra a manutenção
        \nRecebe: 
        \n*Técnico responsável pela manutenção
        \n*Data da manutenção no formato dd-mm-aaaa
        \n*Tensão medida da bateria
        \n*Defeito encontrado na bateria (Não é necessário relatar baixa tensão)
        """
        super().manutencao(tec,data,defeito)
        self._setTensao(tensao)
        nova_medicao = pd.DataFrame({"Data":[self.getDataUltimaManutencao()], 
                                     "Tec":[self.getTecUltimaManutencao()], 
                                     "Defeito":[defeito], 
                                     "Tensao":[self.getTensao()]})
        self._historico_manutencao = pd.concat([self._historico_manutencao, nova_medicao], ignore_index=True)
        self._prox_manutencao = self._proximaManutencao()
        

In [5]:
class FonteTimer(Equipamento):
    """"
    Classe que define uma FonteTimer
    \nFonteTimer são UPS que tem uma bateria interna e recebem um sinal em um entrada de botoeira e 
    \ncomandam um canal de rele por determinado tempo programado nelas.
    \nRecebe: Tudo que um equipamento gêrico tem
    \nRecebe: Bateria, tensão de saída (geralmente 12V)
    """
    def __init__(self,tec,local:str,marca:str,modelo:str,bateria:Bateria,tensao_saida=12,tensao_entrada=127,data_instalacao=None,temp_garantia=None,data_compra=None):
        super().__init__(tipo="FonteTimer",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=tensao_entrada,
                         dc=False,
                         instalador=tec,
                         data_compra=data_compra,
                         data_instalacao=data_instalacao,
                         temp_garantia=temp_garantia)
        #Individuais de uma FonteTimer
        self._bateria = bateria
        self._tensao_saida = tensao_saida
        self._ultima_tensao_saida = self._tensao_saida
        self._ultima_tensao_entrada = self._tensao

    def setBateria(self,bateria:Bateria):
        self._bateria = bateria
    def getBateria(self):
        return self._bateria
    
    def getTensaoSaida(self):
        """Recebe tensão de saída da fonte Timer
        \nTensão usada para carregamento da bateria
        """
        return self._tensao_saida
    
    def getUltimaTensaoEntrada(self):
        return self._ultima_tensao_entrada
    
    def manutencao(self,tec,data,tensao_entrada=None, tensao_saida=None,defeito=None):
        if tensao_entrada: #Se a tensão de entrada foi medida na manutenção
            self._ultima_tensao_entrada = tensao_entrada
        if tensao_saida: #Se a tensão de saída foi medida na manutenção
            self._tensao_saida = tensao_saida
            if self.getTensaoSaida() < self.getBateria().getTensaoCarregamento()[0]: #Se a tensão da fonte timer esta abaixo da tensão de carregamento da bateria
                defeito += f"\n Tensão da fonte timer abaixo da tensão de carregamento da bateria"
            if self.getTensaoSaida() > self.getBateria().getTensaoCarregamento()[1]: #Se a tensão da fonte timer esta acima da tensão de carregamento da bateria
                defeito += f"\n Tensão da fonte timer acima da tensão de carregamento da bateria"
        
        tensao_ref = self.getTensao(self)
        tensao_medida = self.getUltimaTensaoEntrada(self)
        if tensao_ref - tensao_medida > (tensao_ref*0.07) : #Queda de tensão maxima permitida pela NBR5410
            defeito=f"\n Tensão de entrada ({tensao_medida} V) abaixo de {self.getTensao(self)*0.93}V (-7% do previsto)"
        super().manutencao(tec=tec,data=data,defeito=defeito)
            

In [6]:
import re
def valida_ipv4(ip:str):
    """Valida se o ip esta dentro do range valido
    \nRecebe uma string com o ip 
    \nRetorna None se o ip é invalido e o ip se for válido
    """ 
    regex = re.compile(r"^(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])$") 
    return regex.match(ip) is not None

class EquipamentoDeRede:
    """
Classe EquipamentoDeRede
\nA classe define uma equipamento de rede gênerico com ip,mascara de subrede e gateway
    """
    def __init__(self,ip,mascara_sub_rede,gateway):
        """
        Inicializador da classe Equipamento de Rede
        \nRecebe IP, mascara de sub rede e gateway (todos como string)
        """
        if valida_ipv4(ip):
            self._ip = ip
        if valida_ipv4(mascara_sub_rede):
            self._mascara_sub_rede = mascara_sub_rede
        if valida_ipv4(gateway):
            self._gateway = gateway

    def getIp(self):
        """
    Retorna o IP do Equipamento de Rede
    """
        return self._ip
    def setIp(self,ip:str):
        """
Define IP do equipamento
\nRecebe string com ip
        """
        self._ip = valida_ipv4(ip)
    
    def getMascaraSubRede(self):
        return self._mascara_sub_rede
    def setMascaraSubRede(self,mascara_sub_rede):
        self._mascara_sub_rede = valida_ipv4(mascara_sub_rede)

    def getGateway(self):
        return self._gateway
    def setGateway(self,gateway):
        self._gateway = valida_ipv4(gateway) 
       
        

In [7]:
class Leitor(Equipamento,EquipamentoDeRede):
    def __init__(self,nome,marca,modelo,tipo,ip,mascara_sub_rede,gateway,RS485,local,tec,tensao=12,dc=True,temp_garantia=3):
        EquipamentoDeRede.__init__(self,ip,mascara_sub_rede,gateway)
        Equipamento.__init__(self,
                             tipo=tipo,
                             marca=marca,
                             modelo=modelo,
                             local=local,
                             tensao=tensao,
                             dc=dc,
                             temp_garantia=temp_garantia,
                             instalador=tec)
        self._nome = nome #nome do Leitor
        self._RS485 = RS485 #Endereço RS485 do leitor
     
    def getNome(self)->str:
        """
        Retorna o nome dado ao Leitor
        """
        return self._nome
    def setNome(self,nome:str):
        """
        Define o nome do Leitor
        """
        self._nome = nome
    
    def getRS485(self)->int:
        """Retorna endereço RS485"""
        return self._RS485
    def setRS485(self,RS485):
        """Define o endereço RS485 """
        self._RS485 = RS485


In [8]:
class BotoeiraEmergencia(Equipamento):
    def __init__(self,marca,modelo,local,tec,tensao=12,data_compra=None, data_instalacao=None,temp_garantia=3):
        super().__init__(tipo="Botoeira de Emergência",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=tensao,
                         dc=True,
                         temp_garantia=temp_garantia,
                         instalador=tec,
                         data_instalacao=data_instalacao,
                         data_compra=data_compra)
        

In [9]:
class Ima(Equipamento):
    def __init__(self,marca:str,modelo:str,local:str,tensao:int,kgf:int,tec,data_compra=None,data_instalacao=None,temp_garantia=3):
        super().__init__(tipo="Eletroimã",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=tensao,
                         dc=True,
                         instalador=tec,
                         data_compra=data_compra,
                         data_instalacao=data_instalacao,
                         temp_garantia=temp_garantia)
        self._kgf = kgf

    def getKgf(self)->int:
        return self._kgf
    def setKgf(self,kgf:int):
        self._kgf = int(kgf)

In [10]:
class Porta:
    def __init__(self,nome,botoeira_emerg,ima,leitor_entrada,leitor_saida=None):
        self._nome = nome
        self._botoeira_emerg = botoeira_emerg
        self._ima = ima
        self._leitor_entrada = leitor_entrada
        self._leitor_saida = leitor_saida
    def getNome(self):
        return self._nome
    def setNome(self,nome):
        self._nome = nome
    def getBotoeira_emerg(self):
        return self._botoeira_emerg
    def setBoteira_emerg(self,botoeira):
        self._botoeira_emerg = botoeira
    def getIma(self):
        return self._ima
    def setIma(self,ima):
        self._ima = ima
    def getLeitorEntrada(self):
        return self._leitor_entrada
    def setLeitorEntrada(self,leitor):
        self._leitor_entrada = leitor
    def getLeitorSaida(self):
        if self._leitor_saida:
            return self._leitor_entrada
        else:
            return "Não existe leitor de saída"
    def setLeitorSaida(self,leitor):
        self._leitor_entrada = leitor


In [128]:
class Controladora(FonteTimer,Equipamento):
    def __init__(self,local:str,
                 nome:str,
                 marca:str,
                 modelo:str,
                 bateria:Bateria,
                 portas:list,
                 ip:str,
                 mascara_sub_rede:str,
                 gateway:str,
                 tec,
                 tensao_saida=12,
                 tensao_entrada=127,
                 data_instalacao=None,
                 temp_garantia=None,
                 data_compra=None,
                 ):
        FonteTimer.__init__(self=self,
                        local=local,
                         marca=marca,
                         modelo=modelo,
                         bateria=bateria,
                         tensao_saida=tensao_saida,
                         tensao_entrada=tensao_entrada,
                         data_instalacao=data_instalacao,
                         data_compra=data_compra,
                         temp_garantia=temp_garantia,
                         tec=tec)
        EquipamentoDeRede.__init__(self,ip=ip,mascara_sub_rede=mascara_sub_rede,gateway=gateway)
        self._portas = portas
        self._nome = nome

    def getPortas(self)->list[Porta]:
        return self._portas
    def setPortas(self,porta:Porta,indice):
        self._portas[indice] = Porta




In [117]:

# Exemplo de uso
bateria = Bateria(marca="Intelbras")
print(bateria.getTensaoCiclo())  # (13.5, 13.8)

(13.5, 13.8)


In [119]:
fonteTimer_1 = FonteTimer("4GAB","Intelbras",bateria)

In [123]:
fonteTimer_1.getBateria().getUltimaMedicao()

datetime.datetime(2025, 1, 10, 0, 0)

In [116]:
bateria.getUltimaMedicao()

In [122]:
bateria.medicao(tensao=12.1,data="10-01-2025")

In [112]:
bateria.IsForaDaGarantia()

True

In [111]:
bateria._data_compra = datetime.strptime("09-09-2023","%d-%m-%Y")